# Libraries

In [ ]:
import yaml
from pathlib import Path
import tifffile
import os

# Helper Functions

# Main

In [ ]:
config_yaml_path = "config.yaml"

config_path = Path(config_yaml_path)

with config_path.open("r", encoding="utf-8") as f:
    cfg = yaml.safe_load(f) 

data_path = cfg["data"]["path"]


In [ ]:
imgs_shape = {}
for tiff_img_name in os.listdir(data_path):
    if tiff_img_name.endswith(".marker"):
        continue

    tiff_img = tifffile.imread(
        os.path.join(data_path, tiff_img_name)
    )

    shape_name = f"{tiff_img.shape[0]}_{tiff_img.shape[1]}_{tiff_img.shape[2]}"
    if shape_name not in imgs_shape.keys():
        imgs_shape[shape_name] = [tiff_img_name]
    else:
        tiff_img_names = imgs_shape[shape_name]
        updated = tiff_img_names + [tiff_img_name]
        imgs_shape[shape_name] = updated

In [ ]:
from pprint import pprint
pprint(imgs_shape)